    ☐ State class 
    ☐ DiscreteStateSpace 
    ☐ State space representations Regular vs Hamiltonian, State, HamiltonianState 

## **State Space** 

Please stay tuned, here we will have the state space representations of the models...

In [ ]:
from darli.backend import CasadiBackend, PinocchioBackend
from darli.robots import biped
from darli.functional import Functional

# from darli.model import Model

# from robot_descriptions import z1_description

# model = Functional(CasadiBackend(z1_description.URDF_PATH))
# model = Robot(CasadiBackend(z1_description.URDF_PATH))

## **Prespecified Robots**

One can build different robots....

As example let us consider the Atlas humanoid robot:

Note: robot loaded in example is `fixed` in its pelvis and in real world, you have to create a floating base model to have a full set of generalized coordinates.

In [ ]:
from robot_descriptions import atlas_v4_description

biped_urdf = atlas_v4_description.URDF_PATH

biped_model = biped(
    Functional,
    CasadiBackend,
    biped_urdf,
    torso={"torso": "pelvis"},
    foots={
        "left_foot": "l_foot",
        "right_foot": "r_foot",
    },
)

In [ ]:
import numpy as np

In [ ]:
biped_model.nu

In [ ]:
biped_model.nq

In [ ]:
biped_model.body("left_foot").contact.qforce

In [ ]:
biped_model.contact_qforce

In [ ]:
biped_model.inverse_dynamics

In [ ]:
biped_model.body("torso").position

In [ ]:
biped_model.body("torso").rotation

In [ ]:
biped_model.body("torso").jacobian.world_aligned

In [ ]:
biped_model.body("torso").angular_velocity.local

In [ ]:
biped_model.body("torso").jacobian_dt.world_aligned

In [ ]:
biped_model.body("left_foot").contact.jacobian

In [ ]:
biped_model.body("right_foot").contact.jacobian

In [ ]:
q_next = biped_model.backend.integrate_configuration(dt=0.001)

In [ ]:
import casadi as cs

q_next = biped_model.backend.integrate_configuration(dt=cs.SX.sym("dt", 1))

#### **State space Representation and Linearization**

One can easily transform the mechanical system to the state space form by defining the state $\mathbf{x} = [\mathbf{q}, \mathbf{v}]^T$:


$$
\dot{\mathbf{x}}= \mathbf{f}(\mathbf{x}, \mathbf{u}) = 
\begin{bmatrix}
\dot{\mathbf{x}}_1 \\ 
\dot{\mathbf{x}}_2
\end{bmatrix}=
\begin{bmatrix}
\dot{\mathbf{q}} \\ 
\dot{\mathbf{v}}
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{q})\mathbf{v} \\ 
\text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{S}\mathbf{u})
\end{bmatrix}=
\begin{bmatrix}
\mathbf{W}(\mathbf{x}_1)\mathbf{x}_2 \\
\text{aba}(\mathbf{x}_1, \mathbf{x}_2, \mathbf{S}\mathbf{u})
\end{bmatrix}
$$



In [ ]:
model.state_space

The above equation can be easily linearized to produce following linear approximation:

One can easily find linearization with respect to state:

In [ ]:
model.state_space.state_derivative

In [ ]:
model.state_space.state_jacobian

and control:

In [ ]:
model.state_space.input_jacobian

These functionality allows for easy implementation of linearization based analysis and control.

#### **Bodies and Contacts**

In [ ]:
model.add_body(["link06"])
model.bodies

In [ ]:
model.body("link06")

One may also retrieve a hash map of all bodies:

In [ ]:
model.bodies

In [ ]:
model.body("link06").linear_acceleration

In [ ]:
model.body("link06").linear_acceleration.local

In [ ]:
model.body("link06").jacobian.local

In [ ]:
model.body("link06").jacobian.local
model.body("link06").jacobian_dt.local
model.body("link06").linear_velocity.local
model.body("link06").angular_velocity.local
model.body("link06").linear_acceleration.local
model.body("link06").angular_acceleration.local

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

In [ ]:
model.state_space.state_jacobian

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

In [ ]:
model.body("link06").contact.cone.linear()

In [ ]:
model.add_body({"ee": "link06"})
model.bodies.keys()

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.